In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Prose/Mathieu/Benter-Project

/home/mathieu/Prose/Mathieu/Benter-Project


In [2]:
%matplotlib inline
import collections
import functools

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import seaborn as sns
import pandas as pd
from tqdm import tqdm


from utils import import_data
from winning_horse_models.logistic_regression import LogisticRegressionModel
from constants import PMU_BETTINGS, Sources, SplitSets, UnibetBetRateType
from utils.wagering_stategy import compute_expected_return, compute_scenario, plot_scenario, plot_expected_return
import wagering_stategies

from wagering_stategies import factories as wagering_factories

from matplotlib import style
style.use('ggplot')

In [3]:
from winning_horse_models import OddsCombinedWinningModel
winning_model = OddsCombinedWinningModel(alpha=0.3, beta=0.8, winning_model=LogisticRegressionModel.load_model(prefix="48_col_"))

In [4]:
for compute_betting_fun in [wagering_stategies.race_betting_best_expected_return, 
                            wagering_stategies.race_betting_best_winning_proba,
                           wagering_stategies.race_betting_best_winning_proba_not_max_pari_mutual_proba,
                           wagering_stategies.race_betting_proportional_positive_return,
                           wagering_stategies.race_betting_proportional_winning_proba,
                           wagering_stategies.race_bettings_kelly,
                           wagering_stategies.race_least_risky_horse,
                           wagering_stategies.race_proportional_odds,
                           wagering_stategies.race_proportional_pari_mutual_proba,
                           wagering_stategies.race_random_all_horses,
                           wagering_stategies.race_random_one_horse,
                           wagering_stategies.race_rickiest_horse]:
    print(compute_betting_fun.__name__)
    e_r_df = compute_expected_return(
                    compute_betting_fun=compute_betting_fun,
                    source=Sources.UNIBET,
                    code_pari=UnibetBetRateType.SIMPLE_WINNER,
                    winning_model=winning_model,
                ) 
    plot_expected_return(expected_return_df=e_r_df)
    print()

  0%|          | 0/33680 [00:00<?, ?it/s]

race_betting_best_expected_return


/home/mathieu/Prose/Mathieu/Benter-Project/utils/preprocess.py:349: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df["rope_n"] = features_df["rope_n"] / features_df["n_horses"]
/home/mathieu/Prose/Mathieu/Benter-Project/utils/preprocess.py:352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ) / features_df["rope_n"].std()
/home/mathieu/Prose/Mathieu/Benter-Project/utils/preprocess.py:353: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

  0%|          | 7/33680 [01:26<95:48:24, 10.24s/it] 

  0%|          | 8/33680 [01:26<67:35:40,  7.23s/it]

...

  0%|          | 16/33680 [01:28<8:56:51,  1.05it/s] 

  0%|          | 0/33680 [00:00<?, ?it/s]             

On all races, your expected return is +inf% (std: nan)
You bet 0.00% of the time (0 out of 33490 races)
Average not betting streaks: 33490.00 races
When you bet, on average your expected return is +nan% (std: nan)
When you bet, you win nan% of the time (0 out of 0 bets)
When you bet, you lose nan% of the time(0 out of 0 bets)
When you bet&win, you make +inf% (std: nan)
There is no losing streaks!
There is no winning streaks!

race_betting_best_winning_proba


/home/mathieu/Prose/Mathieu/Benter-Project/utils/preprocess.py:349: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df["rope_n"] = features_df["rope_n"] / features_df["n_horses"]
/home/mathieu/Prose/Mathieu/Benter-Project/utils/preprocess.py:352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ) / features_df["rope_n"].std()
/home/mathieu/Prose/Mathieu/Benter-Project/utils/preprocess.py:353: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

AssertionError: 

In [ ]:
%%time
res={}
for minimum_winning_proba in np.arange(0.0, 0.51, 0.1):
    for minimum_expected_return in np.arange(0.0, 16.1, 2):
        e_r_df = compute_expected_return(
                    compute_betting_fun=wagering_factories.betting_on_best_exp_return_thresholded_winning_proba_expected_returns_factory(minimum_winning_probabilities=minimum_winning_proba, 
                                                                                                                                                      expected_return_threshold=minimum_expected_return),
                    source=SOURCE_PMU,
                    code_pari="E_SIMPLE_GAGNANT",
                    winning_model=winning_model,
                ) 
        print(minimum_winning_proba, minimum_expected_return, e_r_df.relative_expected_return.mean(),e_r_df.relative_expected_return.std())
        res[(minimum_winning_proba,minimum_expected_return)]={'mean':e_r_df.relative_expected_return.mean(), 'std':e_r_df.relative_expected_return.std()}
        plot_expected_return(expected_return_df=e_r_df)
        print()

In [ ]:
print(res)

In [ ]:
[(k,v) for k,v in res.items() if v['mean']==max(val['mean'] for val in res.values())]

In [ ]:
sorted(res.items(), key=lambda kv:kv[1]['mean'], reverse=True)

In [ ]:
import pylab
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = pylab.figure()
ax = Axes3D(fig)
X = np.array([k[0] for k in res.keys()])
Y = np.array([k[1] for k in res.keys()])

Z = np.array([v['mean'] for v in res.values()])
ax.set_xlabel('Min. Winning Proba')
ax.set_ylabel('Min. Expected Return')
ax.set_zlabel('Relative Expected Return Mean')

ax.plot_trisurf(X, Y, Z, cmap='hot')

pylab.show()

In [ ]:
fig = pylab.figure()
ax = Axes3D(fig)
X = np.array([k[0] for k in res.keys()])
Y = np.array([k[1] for k in res.keys()])

Z = np.array([v['std'] for v in res.values()])

ax.plot_trisurf(X, Y, Z, cmap='hot')

ax.set_xlabel('Min. Winning Proba')
ax.set_ylabel('Min. Expected Return')
ax.set_zlabel('Relative Expected Return Std')
pylab.show()